## Local Retrieval-Augmented Generation (RAG) chatbot

### 🔽 Step 1: Install Dependencies

In [1]:
pip install pymupdf langchain unstructured

     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     --- ----------------------------------- 92.2/981.5 kB 5.5 MB/s eta 0:00:01
     ------------------- ------------------ 491.5/981.5 kB 6.2 MB/s eta 0:00:01
     -------------------------------------- 981.5/981.5 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/18.5 MB ? eta -:--:--
   -- ------------------------------------- 1.0/18.5 MB 20.7 MB/s eta 0:00:01
   ---- ----------------------------------- 2.0/18.5 MB 20.9 MB/s eta 0:00:01
   ---- ----------------------------------- 2.2/18.5 MB 22.9 MB/s eta 0:00:01
   ------- -------------------------------- 3.3/18.5 MB 17.3 MB/s eta 0:00:01
   -------- ------------------------------- 3.8/18.5 MB 16.2 MB/s eta 0:00:01
   --------- ------------------------------ 4.2/18.5 MB 14.2 MB/s eta 0:00:02
   --------- ------------------------------ 4.4

In [7]:
pip install sentence-transformers

   ---------------------------------------- 0.0/345.7 kB ? eta -:--:--
   -- ------------------------------------ 20.5/345.7 kB 682.7 kB/s eta 0:00:01
   -------------------- ------------------- 174.1/345.7 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 345.7/345.7 kB 3.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install faiss-cpu

   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/15.0 MB 1.6 MB/s eta 0:00:10
   ---------------------------------------- 0.2/15.0 MB 2.2 MB/s eta 0:00:07
   - -------------------------------------- 0.5/15.0 MB 3.5 MB/s eta 0:00:05
   -- ------------------------------------- 1.1/15.0 MB 5.6 MB/s eta 0:00:03
   ---- ----------------------------------- 1.5/15.0 MB 6.4 MB/s eta 0:00:03
   ---- ----------------------------------- 1.7/15.0 MB 6.1 MB/s eta 0:00:03
   ----- ---------------------------------- 1.9/15.0 MB 5.8 MB/s eta 0:00:03
   ----- ---------------------------------- 2.2/15.0 MB 5.7 MB/s eta 0:00:03
   ------ --------------------------------- 2.5/15.0 MB 5.8 MB/s eta 0:00:03
   ------- -------------------------------- 2.8/15.0 MB 6.0 MB/s eta 0:00:03
   --------- ------------------------------ 3.5/15.0 MB 6.7 MB/s eta 0:00:02
   ----------

### 📄 Step 2: Use PyMuPDFLoader to Load PDFs

In [3]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the PDF file
loader = PyMuPDFLoader("Statistics_Notes_and_Practice.pdf")  # Replace with your actual PDF path
documents = loader.load()

# Split the document into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = splitter.split_documents(documents)

### 🧠 Step 3: Create Embeddings and FAISS Vector Store

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Load sentence-transformers embedding model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create FAISS index
db = FAISS.from_documents(docs, embedding_model)

# Save for Streamlit use later
db.save_local("faiss_index")

### 🔁 Step 4: Create Retriever + Prompt Template

In [15]:
retriever = db.as_retriever()

from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Load a local Hugging Face model
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512)
llm = HuggingFacePipeline(pipeline=pipe)

# Build the QA chain
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

C:\Users\nms31\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nms31\.cache\huggingface\hub\models--google--flan-t5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

Device set to use cpu
C:\Users\nms31\AppData\Local\Temp\ipykernel_20588\3397492764.py:14: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


### 💬 Step 5: Ask Questions

In [17]:
query = "What is the document about?"
result = qa_chain(query)

print("Answer:", result["result"])

C:\Users\nms31\AppData\Local\Temp\ipykernel_20588\111526805.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain(query)
Token indices sequence length is longer than the specified maximum sequence length for this model (616 > 512). Running this sequence through the model will result in indexing errors


Answer: To test assumptions (hypotheses) using statistical methods. - Types: - Z-Test - T-Test - Chi-Square Test - ANOVA Errors in Hypothesis Testing: - Type I Error: Rejecting a true null hypothesis (false positive). - Type II Error: Failing to reject a false null hypothesis (false negative). Used in regression for parameter significance and in classification for threshold optimization. Methods and Python Code for Hypothesis Testing 1. Z-Test p-value in a t-Test from scipy.stats import t# Example: T-test t_score = 2.1 # Replace with your calculated t_score degrees_of_freedom = 10 # Replace with your sample's degrees of freedom p_value = 2 * (1 - t.cdf(abs(t_score), df=degrees_of_freedom)) # Two-tailed test print("P-value for T-test:", p_value) The degrees of freedom (df) for a sample are calculated using the formula: df=n1 Where n is the sample size (the total number of observations in the sample).
